In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
np.random.seed(1988)

In [3]:
! dir

 El volumen de la unidad C no tiene etiqueta.
 El número de serie del volumen es: 48E8-1CA4

 Directorio de C:\Users\SynergicTemp\Downloads\4 - Cuisine_categorization

03/07/2018  15:05    <DIR>          .
03/07/2018  15:05    <DIR>          ..
03/07/2018  12:18    <DIR>          .ipynb_checkpoints
26/06/2018  15:53         1.540.094 array.csv
03/07/2018  15:05            22.093 Cuisine - NN.ipynb
02/07/2018  16:09           114.283 Cuisine.ipynb
02/07/2018  15:21           190.706 Kaggle_Lösung.ipynb
25/06/2018  17:23            26.415 sample_submission.csv.zip
13/05/2018  18:21         2.844.086 test.json
13/05/2018  18:21        12.415.067 train.json
               7 archivos     17.152.744 bytes
               3 dirs  394.148.671.488 bytes libres


### Data exploration

In [4]:
data = pd.read_json("./train.json")
data.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [5]:
#there is no NaNs
data[pd.isna(data['ingredients'])]

,cuisine,id,ingredients


In [6]:
data.shape

(39774, 3)

In [7]:
def extract_ingredients(serie):
    list_ingredients=[]
    for lista in serie:
        for element in lista:
            if element in list_ingredients:
                pass
            elif element not in list_ingredients:
                list_ingredients.append(element)
            else:
                pass
        
    return list_ingredients      

In [8]:
ingredients = extract_ingredients(data['ingredients'])

In [9]:
len (ingredients)

6714

In [10]:
#Types of differents cuisines:
data['cuisine'].unique().shape

(20,)

In [11]:
cuisines = data['cuisine'].unique()
cuisines

array(['greek', 'southern_us', 'filipino', 'indian', 'jamaican',
       'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai',
       'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese',
       'irish', 'korean', 'moroccan', 'russian'], dtype=object)

We will try two feature engineering methods, one hot encoding and feature hashing to see what is more effective.

## One hot encoding method

In [12]:
#Create columns
t = time.time()
for ingredient in ingredients:
    data[ingredient]=np.zeros(len(data["ingredients"]))

print("It took %i seg" %(time.time()-t))

It took 154 seg


In [13]:
def ohe(serie):    
    ind=0
    for lista in serie:
        for ingredient in lista:
            if ingredient in ingredients:
                data.loc[ind,ingredient]=1
            else:
                pass
        ind +=1

In [14]:
t = time.time()
ohe(data['ingredients'])
print('it took %i segs' % (time.time()-t))

it took 285 segs


#### Train / test split

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
predictors = ingredients
response = 'cuisine'

In [21]:
data.head()

,cuisine,id,ingredients,romaine lettuce,black olives,grape tomatoes,garlic,pepper,purple onion,seasoning,...,Oscar Mayer Cotto Salami,Challenge Butter,orange glaze,cholesterol free egg substitute,ciabatta loaf,LiptonÂ® Iced Tea Brew Family Size Tea Bags,Hidden ValleyÂ® Greek Yogurt Original RanchÂ® Dip Mix,lop chong,tomato garlic pasta sauce,crushed cheese crackers
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,indian,22213,"[water, vegetable oil, wheat, salt]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
X = data[predictors]
Y = data[response]

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

### Neural Networks

In [24]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

In [17]:
np.random.seed(1988)

In [29]:
len(cuisines)

20

In [42]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = np_utils.to_categorical(encoded_y_train)

In [43]:
encoded_y_train

array([ 9,  2,  6, ...,  3,  9, 16], dtype=int64)

In [44]:
dummy_y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Now we have and output vector, with the categories Ohe

In [60]:
# Create model:
def baseline_model():
    model = Sequential()
    model.add(Dense(100, input_dim=6714, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(20, activation='softmax')) # The output layer must create 20 output values,
    # one for each class. The output value with the largest value will be taken as the class predicted by the model.

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [61]:
from keras.wrappers.scikit_learn import KerasClassifier
estimator = KerasClassifier(build_fn=baseline_model, epochs=5, batch_size=100)

In [63]:
estimator.fit(x_train, dummy_y_train)

Epoch 1/5
31819/31819 [==============================] - 19s 587us/step - loss: 1.3506 - acc: 0.6296
Epoch 2/5
31819/31819 [==============================] - 18s 575us/step - loss: 0.6521 - acc: 0.8108
Epoch 3/5
31819/31819 [==============================] - 19s 592us/step - loss: 0.4832 - acc: 0.8554
Epoch 4/5
31819/31819 [==============================] - 19s 595us/step - loss: 0.3862 - acc: 0.8842
Epoch 5/5
31819/31819 [==============================] - 18s 574us/step - loss: 0.3196 - acc: 0.9048


#### Make predictions:

In [73]:
y_pred = estimator.predict(x_test)
y_pred = encoder.inverse_transform(y_pred)

C:\Program Files\Anaconda\envs\keras_tf\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [48]:
model

Epoch 1/5
31819/31819 [==============================] - 45s 1ms/step - loss: 1.0396 - acc: 0.6992
Epoch 2/5
31819/31819 [==============================] - 51s 2ms/step - loss: 0.5575 - acc: 0.8319
Epoch 3/5
31819/31819 [==============================] - 46s 1ms/step - loss: 0.3964 - acc: 0.8768
Epoch 4/5
31819/31819 [==============================] - 44s 1ms/step - loss: 0.2888 - acc: 0.9107
Epoch 5/5
31819/31819 [==============================] - 49s 2ms/step - loss: 0.2142 - acc: 0.9332


#### Let score the model:

In [46]:
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_y_test = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_test = np_utils.to_categorical(encoded_y_test)

In [71]:
scores = model.evaluate(x_test, dummy_y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

7955/7955 [==============================] - 1s 187us/step

acc: 77.42%


In [72]:
scores

[0.9365877848312886, 0.7742300439750077]